In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetimerange import DateTimeRange

In [8]:
# Raw Data
df = pd.read_csv(r'./Data/DAQ2A.csv')
df.describe()

,outdoor_humidity,outdoor_temperature,cooling_state,furnace_current,furnace_voltage,humid,pump_current,stage_one_H_state,stage_two_H_state,temp,temp_coil
count,173.000000,173.000000,98412.000000,98415.000000,98417.0,98413.0,171915.000000,98413.000000,98411.000000,98413.0,98413.000000
mean,70.265896,2.809249,0.747175,0.757472,120.0,0.0,11.910927,0.026216,0.064586,0.0,18.379167
std,17.096812,4.567775,0.434634,1.295908,0.0,0.0,16.430691,0.159778,0.245796,0.0,9.997943
min,33.000000,-4.000000,0.000000,0.000000,120.0,0.0,8.000000,0.000000,0.000000,0.0,0.000000
25%,57.000000,-1.000000,0.000000,0.200000,120.0,0.0,10.000000,0.000000,0.000000,0.0,17.000000
50%,69.000000,2.000000,1.000000,0.200000,120.0,0.0,10.000000,0.000000,0.000000,0.0,19.000000
75%,85.000000,4.000000,1.000000,0.300000,120.0,0.0,10.000000,0.000000,0.000000,0.0,23.000000
max,99.000000,14.000000,1.000000,5.100000,120.0,0.0,207.000000,1.000000,1.000000,0.0,46.000000


In [ ]:
df1 = df.fillna(0)

In [ ]:
df1["Pump_Power"] = df1['pump_current']*(240)*(0.62)
df1["Furnace_Power"] = df1['furnace_current']*df1['furnace_voltage']*(0.52)
df1.head(30)

In [ ]:
# Change the time-series into datetime
df1['updated_at'] = pd.to_datetime(df1['updated_at'])

In [ ]:
# Extracting the key features
df2 = df1[['updated_at','stage_one_H_state','pump_current','furnace_current','Pump_Power','Furnace_Power']]

In [ ]:
# Filtering where the states are active (1).
Pump_on = df2[(df2['pump_current'] > 0.0) & (df2['furnace_current'] == 0.0)] # Pump only
Furnace_on = df2[(df2['furnace_current'] > 0.0) & (df2['pump_current'] == 0.0)] # Furnace only
Pump_Furnace = df2[(df2['furnace_current'] > 0.0) & (df2['pump_current'] > 0.0)] # Overlap
H1 = df2[(df2['stage_one_H_state'] == 1.0)]

# NB: This doesn't equate to the complete dataset.
count = len(Pump_on) + len(Furnace_on) + len(Pump_Furnace)
count

In [ ]:
# See the data from above: Pump_on, Furnace_on, Pump_Furnace and H1
H1

In [ ]:
# Place data in time-ranges for TOU
# The DAQ samples at every second (1 sec), so to get energy (kWh) it is E = 0.0002 x P
# Choose the dataset: Pump_on, Furnace_on, Pump_Furnace and H1

TOU = {
    'T1': "00:00:00+00:00",
    'T2': "06:59:59+00:00",
    'T3': "07:00:00+00:00",
    'T4': "10:59:59+00:00",
    'T5': "11:00:00+00:00",
    'T6': "16:59:59+00:00",
    'T7': "17:00:00+00:00",
    'T8': "18:59:59+00:00",
    'T9': "19:00:00+00:00",
    'T10': "23:59:59+00:00"
}

All_day = {'T1': "00:00:00+00:00", 'T2':"23:59:59+00:00"}

# Initializing arrays for energy and defining the time range: 16 April 2021 - 23 April 2021
time_range = DateTimeRange("2021-04-16", "2021-04-23")
E1_pump, E2_pump, E3_pump, E4_pump, E5_pump, All_pump  = ([] for i in range(6))
E1_furnace, E2_furnace, E3_furnace, E4_furnace, E5_furnace, All_furnace = ([] for i in range(6)) 
dates = []

for value in time_range.range(datetime.timedelta(days=1)):
    dates.append(value)
    
    start_time = f"{value} {TOU['T1']}"
    end_time = f"{value} {TOU['T2']}"
    mask1 = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table1 = Pump_on.loc[mask1]
    E1_pump.append(((Table1.Pump_Power).sum())*0.0002)
    E1_furnace.append(((Table1.Furnace_Power).sum())*0.0002)

    start_time = f"{value} {TOU['T3']}"
    end_time = f"{value} {TOU['T4']}"
    mask2 = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table2 = Pump_on.loc[mask2]
    E2_pump.append(((Table2.Pump_Power).sum())*0.0002)
    E2_furnace.append(((Table2.Furnace_Power).sum())*0.0002)

    start_time = f"{value} {TOU['T5']}"
    end_time = f"{value} {TOU['T6']}"
    mask3 = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table3 = Pump_on.loc[mask3]
    E3_pump.append(((Table3.Pump_Power).sum())*0.0002)
    E3_furnace.append(((Table3.Furnace_Power).sum())*0.0002)

    start_time = f"{value} {TOU['T7']}"
    end_time = f"{value} {TOU['T8']}"
    mask4 = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table4 = Pump_on.loc[mask4]
    E4_pump.append(((Table4.Pump_Power).sum())*0.0002)
    E4_furnace.append(((Table4.Furnace_Power).sum())*0.0002)

    start_time = f"{value} {TOU['T9']}"
    end_time = f"{value} {TOU['T10']}"
    mask5 = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table5 = Pump_on.loc[mask5]
    E5_pump.append(((Table5.Pump_Power).sum())*0.0002)
    E5_furnace.append(((Table5.Furnace_Power).sum())*0.0002)
    
    # For the whole day for the furnace and pump.
    start_time = f"{value} {All_day['T1']}"
    end_time = f"{value} {All_day['T2']}"
    mask_all = (Pump_on['updated_at'] > start_time) & (Pump_on['updated_at'] <= end_time)
    Table_all = Pump_on.loc[mask_all]
    All_pump.append(((Table_all.Pump_Power).sum())*0.0002)
    All_furnace.append(((Table_all.Furnace_Power).sum())*0.0002)


In [ ]:
Energy_P1 = pd.DataFrame(data=E1_pump, index=dates, columns=["P1_Wh"])
Energy_P2 = pd.DataFrame(data=E2_pump, index=dates, columns=["P2_Wh"])
Energy_P3 = pd.DataFrame(data=E3_pump, index=dates, columns=["P3_Wh"])
Energy_P4 = pd.DataFrame(data=E4_pump, index=dates, columns=["P4_Wh"])
Energy_P5 = pd.DataFrame(data=E5_pump, index=dates, columns=["P5_Wh"])

Energy_F1 = pd.DataFrame(data=E1_furnace, index=dates, columns=["F1_Wh"]) #Midnight-7am
Energy_F2 = pd.DataFrame(data=E2_furnace, index=dates, columns=["F2_Wh"]) #7am - 11am
Energy_F3 = pd.DataFrame(data=E3_furnace, index=dates, columns=["F3_Wh"]) #11am - 5pm
Energy_F4 = pd.DataFrame(data=E4_furnace, index=dates, columns=["F4_Wh"]) #5pm - 7pm
Energy_F5 = pd.DataFrame(data=E5_furnace, index=dates, columns=["F5_Wh"]) #7pm - Midnight

Energy_Furnace = pd.DataFrame(data=All_furnace, index=dates, columns=["Furnace_Wh"])
Energy_Pump = pd.DataFrame(data=All_pump, index=dates, columns=["Pump_Wh"])

In [ ]:
Energy_Pump